In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Author: RenzeLou
# use this script to preprocess all .tsv file to .json format and generate .source_vocab for each split(train/dev/test)
# we don't do lowering because we use bert-base-cased

import json,csv
import numpy as np
import pandas as pd
import re

csv.field_size_limit(500 * 1024 * 1024)

131072

In [2]:
train_file = []
dev_file = []
test_file = []
labels = []

In [3]:
with open("./test_matched.tsv","r",encoding="utf-8") as test:
    t=csv.reader(test,delimiter='\t')
    for k,line in enumerate(t):
        print(list(line))
#         assert len(line) == 4
        if k==2:
            break

['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2']
['0', '31493', '31493', 'travel', '( ( ( ( ( ( ( ( Hierbas , ) ( ans seco ) ) , ) ( ans dulce ) ) , ) and ) frigola ) ( ( ( are just ) ( ( a ( few names ) ) ( worth ( ( keeping ( a look-out ) ) for ) ) ) ) . ) )', '( Hierbas ( ( is ( ( a name ) ( worth ( ( looking out ) for ) ) ) ) . ) )', '(ROOT (S (NP (NP (NNS Hierbas)) (, ,) (NP (NN ans) (NN seco)) (, ,) (NP (NN ans) (NN dulce)) (, ,) (CC and) (NP (NN frigola))) (VP (VBP are) (ADVP (RB just)) (NP (NP (DT a) (JJ few) (NNS names)) (PP (JJ worth) (S (VP (VBG keeping) (NP (DT a) (NN look-out)) (PP (IN for))))))) (. .)))', '(ROOT (S (NP (NNS Hierbas)) (VP (VBZ is) (NP (NP (DT a) (NN name)) (PP (JJ worth) (S (VP (VBG looking) (PRT (RP out)) (PP (IN for))))))) (. .)))', 'Hierbas, ans seco, ans dulce, and frigola are just a few names worth keeping a look-out for.', 'Hierbas is a name wo

In [4]:
with open("./dev_matched.tsv","r",encoding="utf-8") as dev:
    t=csv.reader(dev,delimiter='\t')
    for k,line in enumerate(t):
        print(list(line))
#         assert len(line) == 4
        if k==2:
            break

['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'label2', 'label3', 'label4', 'label5', 'gold_label']
['0', '63735', '63735n', 'slate', '( ( The ( new rights ) ) ( are ( nice enough ) ) )', '( Everyone ( really ( likes ( the ( newest benefits ) ) ) ) )', '(ROOT (S (NP (DT The) (JJ new) (NNS rights)) (VP (VBP are) (ADJP (JJ nice) (RB enough)))))', '(ROOT (S (NP (NN Everyone)) (VP (ADVP (RB really)) (VBZ likes) (NP (DT the) (JJS newest) (NNS benefits)))))', 'The new rights are nice enough', 'Everyone really likes the newest benefits ', 'neutral', 'entailment', 'neutral', 'neutral', 'neutral', 'neutral']
['1', '91383', '91383c', 'government', '( ( This site ) ( ( includes ( ( ( ( a list ) ( of ( all ( award winners ) ) ) ) and ) ( ( a ( searchable database ) ) ( of ( Government ( Executive articles ) ) ) ) ) ) . ) )', '( ( ( The ( Government ( Executive articles ) ) ) ( h

In [5]:
with open("./train.tsv","r",encoding="utf-8") as train:
    t=csv.reader(train,delimiter='\t')
    for k,line in enumerate(t):
        print(list(line))
#         assert len(line) == 4
        if k==2:
            break

['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'gold_label']
['0', '31193', '31193n', 'government', '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )', '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )', '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))', '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))', 'Conceptually cream skimming has two basic dimensions - product and geography.', 'Product and geography are what make cream skimming work. ', 'neutral', 'neutral']
['1', '101457', '101457e', 'telephone'

In [6]:
desolate = 0
with open("./train.tsv","r",encoding="utf-8") as train:
    t=csv.reader(train,delimiter='\t')
    for i,line in enumerate(t):
        if i==0:
            print("skip first line")
            continue
#         print(list(line))
#         try:
#             assert len(line) == 12
#         except:
#             print("error")
# #             print(line)
#             desolate+=1
#             continue
        lb=line[-1]
        if lb not in labels:
            labels.append(lb)
        label = labels.index(lb)
        instance = dict()
        instance["index"],instance['seq1'],instance['seq2'],instance["label"]=int(line[0]),line[8],line[9],label
        train_file.append(instance)

skip first line


In [7]:
desolate

0

In [8]:
with open("./dev_matched.tsv","r",encoding="utf-8") as dev:
    t=csv.reader(dev,delimiter='\t')
    for i,line in enumerate(t):
#         print(list(line))
        if i==0:
            print("skip first line")
            continue
#         print(list(line))
#         try:
#             assert len(line) == 12
#         except:
#             print("error")
#             print(len(line))
#             desolate+=1
#             for p,c in enumerate(line):
#                 print("index:",p," context:",c)
#             continue
        lb=line[-1]
        if lb not in labels:
            labels.append(lb)
        label = labels.index(lb)
        instance = dict()
        instance["index"],instance['seq1'],instance['seq2'],instance["label"]=int(line[0]),line[8],line[9],label
        dev_file.append(instance)

skip first line


In [9]:
with open("./test_matched.tsv","r",encoding="utf-8") as test:
    t=csv.reader(test,delimiter='\t')
    for i,line in enumerate(t):
#         print(list(line))
        if i==0:
            print("skip first line")
            continue
#         print(list(line))
#         try:
#             assert len(line) == 12
#         except:
#             print("error")
#             print(len(line))
#             desolate+=1
#             for p,c in enumerate(line):
#                 print("index:",p," context:",c)
#             continue
#         lb=line[-1]
#         if lb not in labels:
#             labels.append(lb)
#         label = labels.index(lb)
        instance = dict()
        instance["index"],instance['seq1'],instance['seq2'],instance["label"]=int(line[0]),line[8],line[9],None
        test_file.append(instance)

skip first line


In [10]:
dev_file[:10]

[{'index': 0,
  'seq1': 'The new rights are nice enough',
  'seq2': 'Everyone really likes the newest benefits ',
  'label': 0},
 {'index': 1,
  'seq1': 'This site includes a list of all award winners and a searchable database of Government Executive articles.',
  'seq2': 'The Government Executive articles housed on the website are not able to be searched.',
  'label': 2},
 {'index': 2,
  'seq1': "uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him",
  'seq2': 'I like him for the most part, but would still enjoy seeing someone beat him.',
  'label': 1},
 {'index': 3,
  'seq1': "yeah i i think my favorite restaurant is always been the one closest  you know the closest as long as it's it meets the minimum criteria you know of good food",
  'seq2': 'My favorite restaurants are always at least a hundred miles away from my house. ',
  'label': 2},
 {'index': 4,
  'seq1': "i don't know um do you do a lot of camping",

In [11]:
labels

['neutral', 'entailment', 'contradiction']

In [12]:
def simple_filt(seq:str):
    if seq is None:
        return None
    tokens = seq.split()
    tokens = [t for t in tokens if t != ""]
    
    return " ".join(tokens)

In [13]:
def write_to_json(list_files:list,file_names:list,labels:list):
    ''' write cached object to json file(train/dev/test)
    and also generate data statistic information
    
    assert the order of input list is train/dev/test
    '''
    def seq_len(seq:str):
        try:
            return len(seq.split(" "))
        except:
            if seq is None:
                return 0
            else:
                raise KeyError
                
    max_lens = []
    for i,list_file in enumerate(list_files):
        max_len = 0
        new_list = []
        for item in list_file:
            item['seq1'],item['seq2'] = simple_filt(item["seq1"]),simple_filt(item['seq2'])
            max_len_two_seq=max(seq_len(item['seq1']),seq_len(item['seq2']))
            max_len = max(max_len_two_seq,max_len)
            new_list.append(item)
        with open(file_names[i],"w") as f:
            json.dump(new_list,f)
        
        print("successfully dump %s file" %file_names[i])
        max_lens.append(max_len)
    
    data_info = dict()
    train_info = dict()
    dev_info = dict()
    test_info = dict()
    idx2lb = dict()
    for i,lb in enumerate(labels):
        idx2lb[i] = lb
    print("index to label:",idx2lb)
    
    train_info["instance_num"],train_info["instance_max_len"] = len(list_files[0]),max_lens[0]
    dev_info["instance_num"],dev_info["instance_max_len"] = len(list_files[1]),max_lens[1]
    test_info["instance_num"],test_info["instance_max_len"] = len(list_files[2]),max_lens[2]
    
    
    data_info["labels"] = idx2lb
    data_info["train"] = train_info
    data_info["dev"] = dev_info
    data_info["test"] = test_info
    
    data_info_name = "./data_info.json"
    with open(data_info_name,"w") as f:
        json.dump(data_info,f)
    print("successfully dump %s file" %data_info_name )

In [14]:
write_to_json([train_file,dev_file,test_file],["train.json","dev.json","test.json"],labels)

successfully dump train.json file
successfully dump dev.json file
successfully dump test.json file
index to label: {0: 'neutral', 1: 'entailment', 2: 'contradiction'}
successfully dump ./data_info.json file
